# Creation of normal dataset (laurel)

In [7]:
import os
import sys
import numpy as np
import pandas as pd

sys.path.append('../')
sys.path.append('../src/')
sys.path.append('../spell/')

import Reader
import ParamsExtractor3
import DataPreprocessor
import EncodeCommand

In [2]:
log_types = ['laurel']
dates = ['20240418', '20240420', '20240429']
login_node_numbers = ['01', '02', '03', '10']

# Generate the list of file paths
file_paths = [f'/../../../temp_logs/{date}/login{num}.{logtype}.log' for date in dates for num in login_node_numbers for logtype in log_types]

# Filter the list to include only existing files
existing_file_paths = [path for path in file_paths if os.path.exists(path)]
print(existing_file_paths)

df_list = []

for file_path in existing_file_paths:
    r = Reader.Reader(file_path)
    df = r.read_file(file_path)
    df_list.append(df)

['/../../../temp_logs/20240418/login01.laurel.log', '/../../../temp_logs/20240418/login02.laurel.log', '/../../../temp_logs/20240418/login03.laurel.log', '/../../../temp_logs/20240418/login10.laurel.log', '/../../../temp_logs/20240420/login01.laurel.log', '/../../../temp_logs/20240420/login02.laurel.log', '/../../../temp_logs/20240420/login03.laurel.log', '/../../../temp_logs/20240420/login10.laurel.log']


In [3]:
df_after = []

for i, df in enumerate(df_list):
    p = ParamsExtractor3.ParamsExtractor(df)
    df = p.convert_params(df)
    new_df = p.get_params()
    df_after.append(new_df)
    df_list[i] = pd.concat([df_list[i], new_df], axis=1)

In [4]:
for i, df in enumerate(df_list):
    d = DataPreprocessor.DataPreprocessor(df)
    enc = d.drop_and_hash_encode()
    df_list[i] = enc

print(df_list[0].head())

   severity_scores  timedelta    suid  cap_fp      comm      parent_comm  \
0              1.0        0.0  126858       0    squeue             bash   
1              1.0        0.0       0       0      ping  pacemaker-execd   
2              1.0        0.0       0       0  basename             ping   
3              1.0        0.0       0       0     egrep             ping   
4              1.0        0.0       0       0      grep             ping   

   exit  n_dang  n_dang_no_cron  fp_length  ...  col_10  col_11  col_12  \
0     0       2               0         92  ...       0       0       0   
1     0       0               0         38  ...       0       0       0   
2     0       0               0         28  ...       0       0       0   
3     0       0               0         28  ...       0       0       0   
4     0       0               0         21  ...       0       0       0   

   col_13  col_14  col_15  col_16  col_17  col_18  col_19  
0       0       0       0       

In [5]:
for df in df_list:
    print(df.shape)

(1000, 30)
(1000, 30)
(1000, 30)
(1000, 30)
(1000, 30)
(1000, 30)
(1000, 30)
(1000, 30)


In [6]:
df_list[0].head(3)

,severity_scores,timedelta,suid,cap_fp,comm,parent_comm,exit,n_dang,n_dang_no_cron,fp_length,...,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19
0,1.0,0.0,126858,0,squeue,bash,0,2,0,92,...,0,0,0,0,0,0,1,0,0,0
1,1.0,0.0,0,0,ping,pacemaker-execd,0,0,0,38,...,0,0,0,0,0,0,1,0,0,0
2,1.0,0.0,0,0,basename,ping,0,0,0,28,...,0,0,0,0,0,0,1,0,0,0


In [10]:
for i, df in enumerate(df_list):
    
    encoder = EncodeCommand.EncodeCommand()
    
    normal_df_encoded = encoder.encode_command(df)
    normal_df_encoded.fillna(-1, inplace=True)
    df_list[i] = normal_df_encoded
    df_list[i] = df_list[i].drop(columns=['comm', 'parent_comm'], axis=1)
    
    normal_df_encoded.head()

In [11]:
df_list[0].head(3)

,severity_scores,timedelta,suid,cap_fp,exit,n_dang,n_dang_no_cron,fp_length,col_0,col_1,...,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,enc_comm,enc_parent_comm
0,1.0,0.0,126858,0,0,2,0,92,0,0,...,0,0,0,0,1,0,0,0,1,39
1,1.0,0.0,0,0,0,0,0,38,0,0,...,0,0,0,0,1,0,0,0,2,82
2,1.0,0.0,0,0,0,0,0,28,0,0,...,0,0,0,0,1,0,0,0,3,2


In [12]:
logtype = 'laurel_short_'

for i, df in enumerate(df_list):
    filepath = '../data/' + logtype + str(i)
    df.to_csv(filepath, index=False)

# Anomalous dataset creation (laurel)

In [13]:
log_types = ['laurel']

# Generate the list of file paths
file_paths = [f'../data/new_laurel_conf/audit.log' for logtype in log_types]

# Filter the list to include only existing files
existing_file_paths = [path for path in file_paths if os.path.exists(path)]
print(existing_file_paths)

an_df_list = []

for file_path in existing_file_paths:
    r = Reader.Reader(file_path)
    df = r.read_file(file_path)
    print(df.head())
    #df.columns = ['hostname', 'ident', 'message', 'severity', 'facility', 'time']
    an_df_list.append(df)

['../data/new_laurel_conf/audit.log']
                   ID                                            SYSCALL  \
0  1721464625.772:929  {'arch': '0xc000003e', 'syscall': 59, 'success...   
1  1721464625.816:930  {'arch': '0xc000003e', 'syscall': 59, 'success...   
2  1721464625.641:933  {'arch': '0xc000003e', 'syscall': 59, 'success...   
3  1721464625.512:934  {'arch': '0xc000003e', 'syscall': 59, 'success...   
4  1721464625.636:935  {'arch': '0xc000003e', 'syscall': 59, 'success...   

                                              EXECVE                      CWD  \
0  {'argc': 5, 'ARGV': ['/usr/bin/pkla-check-auth...             {'cwd': '/'}   
1  {'argc': 5, 'ARGV': ['/usr/bin/pkla-check-auth...             {'cwd': '/'}   
2  {'argc': 1, 'ARGV': ['/usr/libexec/gsd-rfkill'...  {'cwd': '/var/lib/gdm'}   
3  {'argc': 1, 'ARGV': ['/usr/libexec/gsd-sharing...  {'cwd': '/var/lib/gdm'}   
4  {'argc': 1, 'ARGV': ['/usr/libexec/gsd-keyboar...  {'cwd': '/var/lib/gdm'}   

                  

In [14]:
df = an_df_list[0]
# Extract the numeric part of the timestamp
df['timestamp'] = df['ID'].str.split(':').str[0].astype(float)

# Calculate the difference between consecutive timestamps
df['timedelta'] = df['timestamp'].diff()

In [15]:
def get_params_from_line(line):
    suid, cap_fp, comm, parent_comm, exit = -1,-1,-1,-1,-1

    #print(line[1]['SYSCALL'])
    #print('============')

    line = line[1]
    timedelta = line['timedelta']

    try:
        suid = line['SYSCALL']['suid']
    except:
        pass
    
    try:
        comm = line['SYSCALL']['comm']
    except:
        pass
    
    try:
        exit = line['SYSCALL']['exit']
    except:
        pass
    
    try:
        cap_fp = line['PATH'][0]['cap_fp']
        cap_fp = int(cap_fp, 0)
    except:
        pass
    
    try:
        parent_comm = line['PARENT_INFO']['comm']
    except:
        pass
    
    params = [timedelta, suid, cap_fp, comm, parent_comm, exit, -1, 0, 0, 0]
    #print(params)
    return params

In [16]:
data_list = []
for line in an_df_list[0].iterrows():
    params = get_params_from_line(line)
    data_list.append(params)

an_df = pd.DataFrame(data_list)
an_df.columns = ['timedelta', 'suid', 'cap_fp', 'comm', 'parent_comm', 'exit', 'log key', 'n_dang', 'n_dang_no_cron', 'fp_length']

In [17]:
an_df.head()

,timedelta,suid,cap_fp,comm,parent_comm,exit,log key,n_dang,n_dang_no_cron,fp_length
0,NaN,998,0,pkla-check-auth,-1,0,-1,0,0,0
1,0.044,998,0,pkla-check-auth,-1,0,-1,0,0,0
2,-0.175,42,0,gsd-rfkill,-1,0,-1,0,0,0
3,-0.129,42,0,gsd-sharing,-1,0,-1,0,0,0
4,0.124,42,0,gsd-keyboard,-1,0,-1,0,0,0


In [21]:
encoder = EncodeCommand.EncodeCommand()
anomalous_dataset_encoded = encoder.encode_command(an_df)

anomalous_dataset_encoded = anomalous_dataset_encoded.drop(columns=['comm', 'parent_comm'], axis=1)

total_nans = anomalous_dataset_encoded.isna().sum().sum()
print(f"Total number of NaNs: {total_nans}")

Total number of NaNs: 1


In [22]:
anomalous_dataset_encoded.head(3)

,timedelta,suid,cap_fp,exit,log key,n_dang,n_dang_no_cron,fp_length,enc_comm,enc_parent_comm
0,NaN,998,0,0,-1,0,0,0,1,21
1,0.044,998,0,0,-1,0,0,0,1,21
2,-0.175,42,0,0,-1,0,0,0,2,21


In [23]:
logtype = 'laurel_short_'
filepath = '../data/' + logtype + 'anomalous'
anomalous_dataset_encoded.to_csv(filepath, index=False)